In [1]:
import pandas as pd
import numpy as np

import sys
import os

sys.path.append( "../..")

from global_config import config

results_dir = config.get_property("results_dir")
data_dir    = config.get_property("data_dir")

In [2]:
import requests
import re

seq_urls  = [f"https://raw.githubusercontent.com/nextstrain/dengue/main/example_data/sequences_denv{z}.fasta.zst" for z in range(1, 4+1)]
seq_urls  = seq_urls + [f"https://raw.githubusercontent.com/nextstrain/dengue/main/example_data/sequences_all.fasta.zst"]

meta_urls = [f"https://raw.githubusercontent.com/nextstrain/dengue/main/example_data/metadata_denv{z}.tsv.zst" for z in range(1, 4+1)]
meta_urls = meta_urls + [f"https://raw.githubusercontent.com/nextstrain/dengue/main/example_data/metadata_all.tsv.zst"]

for z, sero_url in enumerate(seq_urls):
    r              = requests.get(sero_url)
    fasta_zst_file = os.path.join(data_dir, "nextstrain", sero_url.split("/")[-1])

    if  os.path.isfile(fasta_zst_file):
        continue

    with open(fasta_zst_file, 'wb') as f:
    #giving a name and saving it in any required format
        f.write(r.content)

    os.system(f"zstd -d {fasta_zst_file}") # un-compress fasta files
    os.system(f"rm -rf {fasta_zst_file}")  # remove compressed fasta files

for z, meta_url in enumerate(meta_urls):
    r             = requests.get(meta_url)
    meta_zst_file = os.path.join(data_dir, "nextstrain", meta_url.split("/")[-1])
    with open(meta_zst_file, 'wb') as f:
    #giving a name and saving it in any required format
        f.write(r.content)
    os.system(f"zstd -d {meta_zst_file}") # un-compress fasta files
    os.system(f"rm -rf {meta_zst_file}")  # remove compressed fasta files

/Users/chaosdonkey06/Dropbox/SantosVega-Lab/dengue_eco_evo/data/nextstrain/sequences_denv1.fasta.zst: 271250 bytes 
/Users/chaosdonkey06/Dropbox/SantosVega-Lab/dengue_eco_evo/data/nextstrain/sequences_denv2.fasta.zst: 304929 bytes 
/Users/chaosdonkey06/Dropbox/SantosVega-Lab/dengue_eco_evo/data/nextstrain/sequences_denv3.fasta.zst: 334582 bytes 
/Users/chaosdonkey06/Dropbox/SantosVega-Lab/dengue_eco_evo/data/nextstrain/sequences_denv4.fasta.zst: 163955 bytes 
/Users/chaosdonkey06/Dropbox/SantosVega-Lab/dengue_eco_evo/data/nextstrain/sequences_all.fasta.zst: 244140 bytes 
/Users/chaosdonkey06/Dropbox/SantosVega-Lab/dengue_eco_evo/data/nextstrain/metadata_denv1.tsv.zst: 6270 bytes 
/Users/chaosdonkey06/Dropbox/SantosVega-Lab/dengue_eco_evo/data/nextstrain/metadata_denv2.tsv.zst: 6104 bytes 
/Users/chaosdonkey06/Dropbox/SantosVega-Lab/dengue_eco_evo/data/nextstrain/metadata_denv3.tsv.zst: 6618 bytes 
/Users/chaosdonkey06/Dropbox/SantosVega-Lab/dengue_eco_evo/data/nextstrain/metadata_denv4

In [3]:
# download and read reference sequences
seq_ref_urls  = [f"https://raw.githubusercontent.com/blab/dengue-antigenic-dynamics/master/data/reference/dengue_{z}_outgroup.gb" for z in range(1, 4+1)]
for url_ref in seq_ref_urls:
    r             = requests.get(url_ref)

    ref_file = os.path.join(data_dir, "nextstrain", url_ref.split("/")[-1])
    with open(ref_file, 'wb') as f:
    #giving a name and saving it in any required format
        f.write(r.content)